In [ ]:
# ------------------------------------------------------------------
# 1. 시각화를 위한 라이브러리 및 데이터셋 준비
# ------------------------------------------------------------------

# 이미지를 파이토치 텐서(Tensor) 형태로 변환하기 위한 도구 정의
transform = transforms.ToTensor()

# 학습용 데이터셋(CIFAR-10) 불러오기
trainset = datasets.CIFAR10(
    root="./data",       # 데이터를 저장할 폴더 위치
    train=True,          # 학습용 데이터(Train)인지 여부 (True면 학습용 5만장)
    download=False,      # 이미 다운로드 받았다면 False, 없으면 True로 설정
    transform=transform  # 이미지를 불러올 때 Tensor로 변환해서 가져옴 (0~255 값을 0~1로 스케일링)
)

classes = trainset.classes         # CIFAR-10의 클래스 이름들 (비행기, 자동차, 새 등 10개)
num_classes = len(classes)         # 클래스 개수 = 10개
images_per_class = 5               # 각 클래스마다 5장씩만 시각화 해보겠다 설정

# 각 클래스별로 이미지를 담을 빈 리스트 딕셔너리 생성 {0: [], 1: [], ... 9: []}
class_to_imgs = {i: [] for i in range(num_classes)}

# 데이터셋을 하나씩 순회하며 이미지와 라벨을 꺼냄
for img, label in trainset:
    # 해당 라벨(클래스)의 이미지 리스트가 아직 5장이 안 찼다면 추가
    if len(class_to_imgs[label]) < images_per_class:
        class_to_imgs[label].append(img)
    
    # 모든 클래스가 5장씩 다 찼다면 반복문 중단 (불필요한 로딩 방지)
    if all(len(v) == images_per_class for v in class_to_imgs.values()):
        break

# 시각화를 위한 캔버스(Figure)와 격자(Axes) 생성
# nrows=10(클래스 수), ncols=5(이미지 수)
fig, axes = plt.subplots(
    nrows=num_classes,
    ncols=images_per_class,
    figsize=(images_per_class * 2, num_classes * 2) # 그림 전체 크기 설정
)

# axes가 1차원 배열일 경우 2차원으로 모양을 맞춰줌 (에러 방지용 안전 장치)
axes = np.array(axes)
if axes.ndim == 1:  
    axes = axes.reshape(num_classes, images_per_class)

# 10개 클래스, 5개 이미지에 대해 반복하며 그림 그리기
for class_idx in range(num_classes):
    for j in range(images_per_class):
        ax = axes[class_idx, j]             # 현재 그림을 그릴 칸 선택
        img = class_to_imgs[class_idx][j]   # 저장해둔 이미지 텐서 가져오기
        
        # [중요 질문 포인트]
        # img.numpy(): 텐서를 넘파이 배열로 변환
        # np.transpose(..., (1, 2, 0)): 차원 순서 변경
        # PyTorch Tensor: (C, H, W) -> (채널, 높이, 너비) ex: (3, 32, 32)
        # Matplotlib imshow: (H, W, C) -> (높이, 너비, 채널) ex: (32, 32, 3) 순서를 원함
        # 따라서 (0, 1, 2) 순서를 (1, 2, 0)으로 바꾸는 과정임
        npimg = np.transpose(img.numpy(), (1, 2, 0)) 

        ax.imshow(npimg) # 이미지 출력
        ax.axis("off")   # x축, y축 눈금 제거 (깔끔하게 보이기 위해)

        # 각 행의 첫 번째 이미지 왼쪽에 클래스 이름(Label) 표시
        if j == 0:
            ax.set_ylabel(
                classes[class_idx],
                rotation=0,      # 글자 가로로 쓰기
                labelpad=40,     # 그림과 글자 사이 여백
                fontsize=10,
                va=\"center\"
            )

plt.tight_layout() # 그림들이 겹치지 않게 레이아웃 자동 조정
plt.show()         # 화면에 출력

In [ ]:
# ------------------------------------------------------------------
# 2. 모델 학습을 위한 데이터 로더 설정
# ------------------------------------------------------------------

# 랜덤 시드 고정 (실행할 때마다 결과가 달라지지 않게 하기 위함)
torch.manual_seed(42)

# GPU 사용 가능하면 'cuda', 아니면 'cpu' 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# 데이터 전처리 파이프라인 정의 (Compose로 묶음)
transform = transforms.Compose([
    transforms.ToTensor(), # 1. 이미지를 0~1 사이의 값을 가진 Tensor로 변환 (H,W,C) -> (C,H,W)
    
    # 2. 정규화 (Normalization)
    # 공식: (pixel_value - mean) / std
    # 아래 값들은 CIFAR-10 데이터셋의 RGB 채널별 평균과 표준편차임 (이미 알려진 값)
    # 이렇게 하면 데이터 분포가 평균 0, 표준편차 1에 가깝게 변해서 학습이 잘 됨
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],  
        std=[0.2470, 0.2435, 0.2616]    
    ),
])

# 학습용 데이터셋 로드 (위에서 정의한 transform 적용)
train_dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

# 테스트용 데이터셋 로드 (train=False)
test_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

# 배치 사이즈 설정 (한 번에 학습할 이미지 개수)
# 128장은 GPU 메모리에 적당히 들어가면서 학습 속도도 빠름
batch_size = 128

# 학습용 데이터 로더
# DataLoader는 데이터를 batch_size만큼 묶어서 모델에 공급하는 역할
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,   # [중요] 학습 데이터는 순서를 섞어야(Shuffle) 모델이 순서를 외우지 않음
    num_workers=0   # 데이터를 로드할 때 사용할 CPU 프로세스 수 (0이면 메인 프로세스만 사용)
)

# 테스트용 데이터 로더
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,  # 테스트 데이터는 섞을 필요 없음 (평가만 하면 되니까)
    num_workers=0
)

classes = train_dataset.classes
print("Classes:", classes)

In [ ]:
import torch
import torch.nn as nn

# ------------------------------------------------------------------
# 3. Convolution Block 정의 (반복되는 구조를 클래스로 만듦)
# ------------------------------------------------------------------
class ConvBlock(nn.Module):
    def __init__(self, channels, use_residual: bool = False):
        super().__init__() # 부모 클래스(nn.Module) 초기화
        self.use_residual = use_residual # Residual Connection 사용 여부 저장
        
        # 첫 번째 합성곱 레이어
        # channels: 입력 채널 수 = 출력 채널 수 (입출력 크기 동일하게 유지하려는 의도)
        # kernel_size=3: 3x3 필터 사용
        # padding=1: 3x3 필터를 쓰면 이미지가 줄어드는데, padding=1을 주면 크기가 유지됨 (32x32 -> 32x32)
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        
        # 두 번째 합성곱 레이어 (구조 동일)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        
        # 활성화 함수 ReLU (음수는 0으로, 양수는 그대로)
        # inplace=True: 메모리 절약을 위해 입력을 직접 수정
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        # x: 입력 데이터
        
        # 1. 첫 번째 Conv 통과 -> ReLU
        out = self.conv1(x)
        out = self.relu(out)
        
        # 2. 두 번째 Conv 통과
        out = self.conv2(out)
        
        # [핵심 질문 포인트] Residual Connection (Skip Connection)
        # use_residual이 True면, "컨볼루션을 통과하기 전의 입력 x"를 "통과한 후의 결과 out"에 더함.
        # 이것이 ResNet의 핵심 아이디어로, 깊은 망에서도 학습이 잘 되게 함(기울기 소실 방지).
        if self.use_residual:
            out += x  
            
        # 3. 마지막에 ReLU 한 번 더 적용
        out = self.relu(out)
        return out

In [ ]:
# ------------------------------------------------------------------
# 4. 전체 CNN 모델 정의 (Residual 없는 버전)
# ------------------------------------------------------------------
class SimpleCNN_NoRes(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        
        # [Layer 1]
        # 입력 이미지: (Batch, 3, 32, 32) -> RGB 3채널
        # Conv2d: 채널을 3 -> 32로 늘림. (특징 32개를 뽑겠다)
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        
        # [Block 1]
        # 위에서 만든 ConvBlock 사용 (Residual 없음)
        # 채널 수는 32로 유지, 이미지 크기 32x32 유지
        self.block1 = ConvBlock(32, use_residual=False)
        
        # [Pooling 1]
        # MaxPool2d(2, 2): 이미지 가로, 세로를 절반으로 줄임 (중요한 정보만 남김)
        # 크기 변화: 32x32 -> 16x16
        self.pool1 = nn.MaxPool2d(2, 2)
        
        # [Layer 2]
        # Conv2d: 채널을 32 -> 64로 두 배 늘림 (더 복잡한 특징 추출)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        
        # [Pooling 2]
        # 크기 변화: 16x16 -> 8x8
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # [Flatten]
        # 2차원 이미지(채널 포함 3차원)를 1차원 벡터로 쫙 폅니다.
        # (Batch, 64, 8, 8) -> (Batch, 64*8*8) -> (Batch, 4096)
        self.flatten = nn.Flatten()
        
        # [Classifier] 분류기 (완전 연결 계층, Fully Connected Layer)
        self.classifier = nn.Sequential(
            nn.Linear(64 * 8 * 8, 256), # 4096개 입력을 받아 256개로 압축
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes) # 256개를 받아 최종 10개(클래스 수) 점수 출력
        )

    # 데이터가 흘러가는 순서 정의
    def forward(self, x):
        x = self.layer1(x)  # (3, 32, 32) -> (32, 32, 32)
        x = self.block1(x)  # (32, 32, 32)
        x = self.pool1(x)   # (32, 32, 32) -> (32, 16, 16)
        
        x = self.layer2(x)  # (32, 16, 16) -> (64, 16, 16)
        x = self.pool2(x)   # (64, 16, 16) -> (64, 8, 8)
        
        x = self.flatten(x) # (64, 8, 8) -> (4096)
        x = self.classifier(x) # (4096) -> (10)
        return x

# ------------------------------------------------------------------
# 5. 전체 CNN 모델 정의 (Residual 있는 버전)
# ------------------------------------------------------------------
class SimpleCNN_Res(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        
        # 구조는 NoRes와 99% 동일함.
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        
        # [차이점] use_residual=True 로 설정하여 Skip Connection 활성화
        self.block1 = ConvBlock(32, use_residual=True)
        
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.flatten = nn.Flatten()
        
        self.classifier = nn.Sequential(
            nn.Linear(64 * 8 * 8, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.block1(x) # 여기 내부에서 더하기(+) 연산이 일어남
        x = self.pool1(x)
        
        x = self.layer2(x)
        x = self.pool2(x)
        
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [ ]:
# ------------------------------------------------------------------
# 6. 학습 및 평가를 수행하는 함수 정의
# ------------------------------------------------------------------
def train_and_evaluate(model, model_name, train_loader, test_loader, device):
    print(f"\n=== Training {model_name} ===")
    
    # 모델을 GPU(또는 CPU) 메모리로 이동시킴
    model = model.to(device)
    
    # Loss Function: CrossEntropyLoss (분류 문제의 정석)
    # 내부적으로 Softmax와 Log-Likelihood Loss가 결합되어 있음
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer: Adam (가장 많이 쓰이는 최적화 알고리즘)
    # lr=1e-3: 학습률 (한 번에 얼마나 수정할지)
    # weight_decay=5e-4: L2 정규화 (가중치가 너무 커지는 것을 방지하여 과적합 억제)
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
    
    epochs = 10 # 전체 데이터셋을 10번 반복해서 학습하겠다
    
    for epoch in range(epochs):
        # --- [1] 학습 모드 (Train) ---
        model.train() # 모델에게 "지금은 학습 중이야"라고 알림 (Dropout, BatchNorm 등이 다르게 동작함)
        running_loss = 0.0 # 에폭당 누적 손실값 초기화
        correct = 0        # 맞춘 개수 초기화
        total = 0          # 전체 데이터 개수 초기화
        
        # train_loader에서 배치 단위로 데이터를 가져옴 (inputs: 이미지, labels: 정답)
        for inputs, labels in train_loader:
            # 데이터를 GPU로 이동
            inputs, labels = inputs.to(device), labels.to(device)
            
            # [중요] 이전 배치의 기울기(Gradient) 초기화
            # 이걸 안 하면 기울기가 계속 누적되어 학습이 엉망이 됨
            optimizer.zero_grad()
            
            # 1. Forward (순전파): 모델에 이미지를 넣어 예측값(outputs) 계산
            outputs = model(inputs)
            
            # 2. Loss Calculation: 예측값과 정답(labels) 비교하여 손실 계산
            loss = criterion(outputs, labels)
            
            # 3. Backward (역전파): 손실에 대한 각 가중치의 기울기 계산
            loss.backward()
            
            # 4. Step: 계산된 기울기를 이용해 가중치 업데이트 (실제 학습이 일어나는 순간)
            optimizer.step()
            
            # 통계 계산을 위해 손실값 누적
            running_loss += loss.item() * inputs.size(0)
            
            # 정확도 계산
            # torch.max(outputs, 1): 확률이 가장 높은 클래스의 인덱스(predicted)를 찾음
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0) # 전체 개수 증가
            correct += (predicted == labels).sum().item() # 정답과 같으면 맞춘 개수 증가
            
        # 에폭별 평균 손실과 정확도 계산
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total * 100
        
        # --- [2] 평가 모드 (Test) ---
        model.eval() # 모델에게 "지금은 평가 중이야"라고 알림 (학습 때만 쓰는 기능들 끔)
        test_loss = 0.0
        correct = 0
        total = 0
        
        # [중요] 평가 때는 미분(Gradient) 계산이 필요 없음 -> 메모리 절약, 속도 향상
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                # Forward (예측)만 수행
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # 테스트 손실 누적
                test_loss += loss.item() * inputs.size(0)
                
                # 테스트 정확도 계산
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
        test_epoch_loss = test_loss / len(test_loader.dataset)
        test_epoch_acc = correct / total * 100
        
        # 결과 출력 (에폭마다 Train/Test 결과를 한 줄에 출력)
        print(f"[{model_name}] Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}% | "
              f"Test Loss: {test_epoch_loss:.4f}, Test Acc: {test_epoch_acc:.2f}%")

# --- 실행 부분 ---

# 1. Residual이 없는 모델 생성 및 학습
model_nores = SimpleCNN_NoRes(num_classes=10)
train_and_evaluate(model_nores, "NoRes", train_loader, test_loader, device)

# 2. Residual이 있는 모델 생성 및 학습 (성능 비교용)
model_res = SimpleCNN_Res(num_classes=10)
train_and_evaluate(model_res, "Res", train_loader, test_loader, device)